In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("http://localhost:5000/")

driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(1) > label > input[type=text]').send_keys("인천국제공항")
driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(2) > label > input[type=text]').send_keys("간사이 국제 공항")
driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(3) > label > input[type=text]').send_keys("오사카")
driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(4) > label > input[type=text]').send_keys(1)
driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(5) > label > input[type=text]').send_keys(1)
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(6) > label > input[type=date]').send_keys("2023")
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(6) > label > input[type=date]').send_keys("12")
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(6) > label > input[type=date]').send_keys("01")
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(7) > label > input[type=date]').send_keys("2023")
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(7) > label > input[type=date]').send_keys("12")
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(7) > label > input[type=date]').send_keys("07")
# driver.find_element(By.CSS_SELECTOR, 'body > form > p:nth-child(8) > input[type=submit]').click()


In [20]:
from flask import Flask, render_template, request

from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import datetime as dt

In [21]:
def hotel(result):
    driver = webdriver.Chrome()
    driver.get(
        "https://www.agoda.com/ko-kr/search?city=14690&site_id=1891442&tag=782b98b0-e95a-0318-464c-9df2f46883a7&nhakw=1&gad=1&device=c&network=g&adid=577294658636&rand=10811873334794923270&expid=&adpos=&aud=aud-951004860236%3Akwd-321247717620&gclid=Cj0KCQjw-pyqBhDmARIsAKd9XINVq9Dyz2VhSt_UehLOyXgVAz8np5YIvOBgizfG7qqCnMH8aFm7VMQaAhz6EALw_wcB&checkIn=2023-11-15&checkOut=2023-11-16&adults=1&rooms=1&pslc=1&ds=cJadpOCWpF1Ik%2FR3"
    )
    search_box = driver.find_element(
        By.XPATH, "//*[@id='autocomplete-box']/div/div/div/div[1]"
    )

    search_box.click()
    time.sleep(2)
    search = driver.find_element(By.XPATH, '//*[@id="textInput"]')
    search.clear()
    search.send_keys(result["place"])
    time.sleep(1)

    print(type(result["start_date"]))
    print(type(result["end_date"]))
    # 날짜
    start_date = change_to_date(result["start_date"])
    end_date = change_to_date(result["end_date"])
    print(start_date)
    print(end_date)
    # test
    driver.find_element(By.CSS_SELECTOR, "#check-in-box").click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '[aria-label="{}"]'.format(start_date)).click()
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, '[aria-label="{}"]'.format(end_date)).click()
    time.sleep(2)

    # 인원입력칸 클릭
    driver.find_element(By.XPATH, '//*[@id="occupancy-box"]/div').click()
    time.sleep(2)

    # 성인 증가
    for i in range(int(result["adult"]) - 1):
        driver.find_element(
            By.XPATH, '//*[@id="occupancy-selector"]/div/div/div[2]/div[2]/div[3]'
        ).click()
        time.sleep(2)

    # 아동 증가
    # ////*[@id="occupancy-selector"]/div/div/div[3]/div[2]/div[3]/svg/use
    for i in range(int(result["Infant"])):
        driver.find_element(By.CSS_SELECTOR, '#occupancy-selector > div > div > div:nth-child(2) > div.sc-bdfBwQ.sc-gsTCUz.jImVoD > div.sc-bdfBwQ.sc-gsTCUz.bAJgph > svg > use').click()
        time.sleep(2)

    driver.find_element(
        By.XPATH, '//*[@id="SearchBoxContainer"]/div/div/button'
    ).click()
    time.sleep(5)

    max_try = 3

    for _ in range(max_try):
        element_to_scroll_to = driver.find_element(
            By.XPATH,
            "/html",
        )
        element_to_scroll_to.send_keys(Keys.PAGE_DOWN)
        element_to_scroll_to.send_keys(Keys.PAGE_DOWN)
        element_to_scroll_to.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    name = driver.find_elements(By.CSS_SELECTOR, "h3.sc-jrAGrp.sc-kEjbxe.eDlaBj.dscgss")
    review = driver.find_elements(
        By.CSS_SELECTOR,
        "p.Typographystyled__TypographyStyled-sc-j18mtu-0.Hkrzy.kite-js-Typography",
    )
    elements_with_class = driver.find_elements(
        By.CLASS_NAME, "sc-jrAGrp.sc-kEjbxe.eDlaBj.fRhhIV.sc-dQppl.cQZIoF"
    )

    position = []

    for element in elements_with_class:
        aria_label = element.get_attribute("label")
        if aria_label:
            position.append(aria_label)
    name_lst = []
    for i in range(int(len(name))):
        name_lst.append(name[i].text)

    review_lst = []
    for i in range(int(len(review))):
        review_lst.append(review[i].text)
    review_lst.pop(0)
    data = {"숙소": name_lst, "평점": review_lst, "위치": position}
    df = pd.DataFrame(data)

In [ ]:
hotel()